In [7]:
# pip install xlrd

In [8]:
"""Generates and submits an email as a draft.
"""
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import urllib.request
import urllib.error

import base64
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import mimetypes
import os

from apiclient import errors


def CreateDraft(service, user_id, message_body):
  """Create and insert a draft email. Print the returned draft's message and id.

  Args:
    service: Authorized Gmail API service instance.
    user_id: User's email address. The special value "me"
    can be used to indicate the authenticated user.
    message_body: The body of the email message, including headers.

  Returns:
    Draft object, including draft id and message meta data.
  """
  try:
    message = {'message': message_body}
    draft = service.users().drafts().create(userId=user_id, body=message).execute()

    print('Successfully executed')

    return draft
  except urllib.error.HTTPError as e:
    print(e)
    return None

In [9]:

def CreateMessage(sender, to, subject, message_text):
  """Create a message for an email.

  Args:
    sender: Email address of the sender.
    to: Email address of the receiver.
    subject: The subject of the email message.
    message_text: The text of the email message.

  Returns:
    An object containing a base64url encoded email object.
  """
  message = MIMEText(message_text)
  message['to'] = to
  message['from'] = sender
  message['subject'] = subject
  b64_bytes = base64.urlsafe_b64encode(message.as_bytes())
  b64_string = b64_bytes.decode()
  return {'raw': b64_string}


def CreateMessageWithAttachment(sender, to, subject, message_text, file_dir,
                                filename):
  """Create a message for an email.

  Args:
    sender: The email address of the sender.
    to: The email address of the receiver.
    subject: The subject of the email message.
    message_text: The text of the email message.
    file_dir: The directory containing the file to be attached.
    filename: The name of the file to be attached.

  Returns:
    An object containing a base64url encoded email object.
  """
  message = MIMEMultipart()
  message['to'] = to
  message['from'] = sender
  message['subject'] = subject

  msg = MIMEText(message_text)
  message.attach(msg)

  path = os.path.join(file_dir, filename)
  content_type, encoding = mimetypes.guess_type(path)

  if content_type is None or encoding is not None:
    content_type = 'application/octet-stream'
  main_type, sub_type = content_type.split('/', 1)
  if main_type == 'text':
    fp = open(path, 'rb')
    msg = MIMEText(fp.read(), _subtype=sub_type)
    fp.close()
  elif main_type == 'image':
    fp = open(path, 'rb')
    msg = MIMEImage(fp.read(), _subtype=sub_type)
    fp.close()
  elif main_type == 'audio':
    fp = open(path, 'rb')
    msg = MIMEAudio(fp.read(), _subtype=sub_type)
    fp.close()
  else:
    fp = open(path, 'rb')
    msg = MIMEBase(main_type, sub_type)
    msg.set_payload(fp.read())
    fp.close()

  msg.add_header('Content-Disposition', 'attachment', filename=filename)
  message.attach(msg)

  return {'raw': base64.urlsafe_b64encode(message.as_string())}

In [10]:
import xlrd

## EDIT THIS FUNCTION!
def sendDrafts(service):
    wb = xlrd.open_workbook("./data/july162019/blast_1_col2.xlsx");
    ws = wb.sheet_by_index(0)
    companies = ws.col_values(0)
    emails = ws.col_values(1)

    for i in range(len(companies)):
        
        message_text = ("Hello!\n\nHope you're doing well! My name is Gouttham Chandrasekar, and I am the Sponsorship Director of Duke Undergraduate Machine Learning, "
        "an undergraduate organization at Duke University that promotes machine learning and related fields at Duke through "
        "events such as workshops, talks, and competitions. \n\nWe're hosting our second annual Duke Datathon, a student-run "
        "data science competition. On November 2-3, 2019, we're bringing in 300+ undergraduate and graduate students to "
        "attack datasets and come up with interesting analyses and visualizations. We are also inviting 100+ students from other schools to participate in the competition. Past sponsors include Google, Pinterest, "
        "Jane Street, IMC Trading, Capital One, and more; members of our organizing team have headed to companies such as Google, "
        "Facebook, Microsoft, AQR Capital, Point72, Apple, and BlackRock. \n\nI'd love the opportunity to tell you more about "
        "the event and see how we can work together to get {0} out in front of some of the top student data scientists, "
        "engineers, and mathematicians from Duke and the east coast. What does your calendar look like for a brief chat this week?"
        "\n\nMore information about sponsoring can be found at dukeml.org/sponsor. Previous events we've hosted include Duke "
        "Datathon during Fall 2018, Machine Learning Days during Spring 2018 and 2019, and our MLBytes speaker series. "
        "More information about our student organization can be found at dukeml.org. \n\nPlease let me know if you have "
        "any other questions! Looking forward to hearing back from you soon.\n\nThanks,\nGouttham Chandrasekar").format(companies[i])
        
        message = CreateMessage("hello@dukeml.org", emails[i], companies[i] + " + Duke Machine Learning", message_text)
    
        CreateDraft(service, 'me', message);

    


In [11]:
SCOPES = ['https://mail.google.com/']
def main():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server()
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)
    sendDrafts(service);
    

    
main();


Successfully executed
Successfully executed
Successfully executed
Successfully executed
Successfully executed
Successfully executed
Successfully executed
Successfully executed
Successfully executed
Successfully executed
Successfully executed
Successfully executed
Successfully executed
Successfully executed
Successfully executed
Successfully executed
Successfully executed
Successfully executed
Successfully executed
Successfully executed
Successfully executed
Successfully executed
Successfully executed
Successfully executed
Successfully executed
